In [ ]:
# improt game + joypad wrapper + nes_py
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

We are using SIMPLE_MOVEMENT, to reduce the complexity of the learning process.
SIMLE_MOVEMENT haes a total of 7 actions instead of 256 (base game), which are:

    [['NOOP'], (no operation)
    ['right'],
    ['right', 'A'],
    ['right', 'B'],
    ['right', 'A', 'B'],
    ['A'],
    ['left']]

In [ ]:
# Set up the game
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode='human')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

The game we are choosing to set up is the followng :

![SuperMarioBros-v0](Images/SuperMarioBros-v0.png)

In [ ]:
# this block's purpose is to test the base game and see if it works
# create a flag to check if the game is done
done = True
for step in range(5):
    if done:
        env.reset()
    
    # Sample a random action
    action = env.action_space.sample()
    # Take a step in the environment
    output = env.step(action)
    
    # Manually append 'False' for 'truncated' if necessary
    if len(output) == 4:  # Only if the output has 4 values, append 'False' for 'truncated'
        output = output + (False,)
    
    # Unpack all five expected values
    state, reward, done, truncated, info = output
    
    # Render the game
    env.render()

# Close the environment
env.close()


## Preprocess Environment

In [ ]:
# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [ ]:
class CustomJoypadSpace(JoypadSpace):
    def reset(self, **kwargs):
        # Ignore 'seed' and 'options' if they are in kwargs
        kwargs.pop('seed', None)
        kwargs.pop('options', None)
        return super().reset(**kwargs)

# Use the custom wrapper
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = CustomJoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
# We now create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode='human')
env = CustomJoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4, channels_order='last')

In [ ]:
state = env.reset()

In [ ]:
state, reward, done, info = env.step([5])

In [ ]:
plt.figure(figsize=(10, 10))
for idx in range(state.shape[3]):
    plt.subplot(1, 4, idx+1)
    plt.imshow(state[0, :, :, idx], cmap='gray')
    plt.axis('off')
plt.show()

## Train the RL model

In [ ]:
# import os and stable baselines
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
# Setup model saving callback
callback = TrainingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)

In [ ]:
# model handling
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)


In [ ]:
model.learn(total_timesteps=5000000, callback=callback)

## Test the RL model

In [ ]:
model = PPO.load(CHECKPOINT_DIR + 'best_model_2500000', env=env)

In [ ]:
state = env.reset()
while True:
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()